Runtime analysis desired takeaway:
- illustrate added processing time from detection association (really big when lots of predictions/weak filtering)

Runtime analysis approach:
- Was runtime information collected during results generation?
    - No; folders and json were all created at same time (in results folder)
- Estimate runtime without running on everything (1 passthrough test set?)
    - For 20/75/88: runtime - w/ assoc vs. w/o assoc
    - For 20/75/88: N detections - w/assoc vs. w/o assoc
    

In [ ]:
from typing import List, Dict, OrderedDict, Union

from afdme import REPO_PATH
from afdme.filter import common, dft
from afdme.run import run
from afdme.data import DataLoader, Dataset

In [ ]:

def multirun_exp(param_batches, args):

    # create the dataset - do this once, do dataloader.reset() each batch
    data_path = ""
    label_path = ""
    if args["dataset"] == "test":
        data_path = f"{args['path']}/mp4/batched_test"
        label_path = f"{args['path']}/labels/cvat-video-1.1/batched_test"
    else:
        data_path = f"{args['path']}/mp4/{args['dataset']}"
        label_path = f"{args['path']}/labels/cvat-video-1.1/{args['dataset']}"
    dataloader = DataLoader(
        Dataset(videos=data_path, labels=label_path), split=args["dataset"]
    )

    for itr, params in enumerate(param_batches):

        fps = 10
        frame_delay = 1.0 / fps

        # reinitialize filters given params
        mean_filter = common.MeanFilter(fps=fps, params=params["mean_filter"])
        turbine_filter = dft.DFTFilter(fps=fps, params=params["turbine_filter"])
        denoise_filter = common.GaussianBlurDenoiseFilter(
            fps=fps, params=params["denoise_filter"]
        )
        intensity_filter = common.IntensityFilter(
            fps=fps, params=params["intensity_filter"]
        )
        contour_filter = common.ContourFilter(params=params["contour_filter"])
        tracklet_association = common.TrackletAssociation(
            params=params["tracklet_association"]
        )

        # automatically load filters at runtime and ensure required filters are present
        # ie {"original": None} and {"contour_filter": contour_filter}
        filters = OrderedDict()
        filters.update({"original": None})
        # NOTE: I think, but am not sure all these filters are order-agnostic
        for filter in args["filters"]:
            try:
                filters.update({filter: eval(filter)})
            except SyntaxError:
                raise SyntaxError(
                    "The only valid filters to pass to --filters are: "
                    + "[mean_filter, turbine_filter, "
                    + "denoise_filter, intensity_filter, "
                    + "tracklet_association"
                )
        # always end with contour_filter
        filters.update({"contour_filter": contour_filter})
        # unless tracklet_association present
        if "tracklet_association" in filters.keys():
            filters.move_to_end("tracklet_association")

        # hardcoded equivalent
        # filters = OrderedDict(
        #    [
        #        ("original", None),
        #        ("mean_filter", mean_filter),
        #        ("turbine_filter", turbine_filter),
        #        ("denoise_filter", denoise_filter),
        #        ("intensity_filter", intensity_filter),
        #        ("contour_filter", contour_filter),
        #        #            ("tracklet_association", tracklet_association),
        #    ]
        # )

        # set self_idx to 0 to restart runs through dataset
        dataloader.reset()

        # run the filters on every video in the dataset; or on N=<max_vid_itrs> videos (for testing)
        run(
            itr,
            filters,
            params,
            dataloader,
            run_path,
            max_vid_itrs=10000000,
        )